In [1]:
import glob
import codecs
import requests
import os
import requests
import datetime as dt
import json
import re
import time
import pandas as pd

In [2]:
def get_device_ID(prefix, sequence):
    url = f"https://devices.senslabs.io/api/recorders/get?sequence={sequence}&prefix={prefix}"

    payload={}
    headers = {
      'x-api-key': 'R4SFERU4m06IJc3jwfAHP228aMkcLeFBapNxyQRT'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    response = response.json()
    response = pd.DataFrame(response)
    return response.RecorderId[0]
def get_raw_files(DeviceId, Emails, From, To):
    url = "https://console.senslabs.io/api/rawfiles/request"
    payload = json.dumps({
        "DeviceId": DeviceId ,
        "From": From ,
        "To": To ,
        "Emails": Emails
        })
    headers = {
        'x-api-key': 'TnxV3x7XOku8T1y4Zqvb',
        'Content-Type': 'application/json'
        }
    response = requests.request("POST", url, headers=headers, data=payload, timeout=5)
    return response


In [3]:
def read_data(file_name):
    asciiFile = codecs.open(file_name, "rb")
    data = asciiFile.read()
    raw_data = []
    for value in data:
        raw_data.append(value)

    asciiFile.close()
    header_position = [i for i, ltr in enumerate(raw_data) if ltr == 252 or ltr == 253]
    header = raw_data[header_position[0]:header_position[1] + 1]
    header = header[1:-1]
    header_str = "".join(chr(x) for x in header)
    header_tstamp = int(header_str[:10])
    return raw_data, header_position, header_tstamp

def update_device_id_htime(raw_data, header_position, device_id, file_name, header_tstamp):
    new_device = [ord(x) for x in str(device_id)]
    new_header = [ord(x) for x in str(header_tstamp)]
    for i in range(len(new_device)):
        raw_data[header_position[0] + 11 + i] = new_device[i]
    for i in range(len(new_header)):
        raw_data[header_position[0] + 1 + i] = new_header[i]

    with open(file_name, "wb") as f:
        data = f.write(bytearray(raw_data))
    f.close()

def upload_file(device_id, recording_id, filename):
    headers = {'Content-Type': 'application/octet-stream',
               'x-api-key': 'daq0:Q0bGUVEoCrUFbuCXcM6c'
               }
    
    # master = "https://fwrest.senslabs.io/api/uploads/0-0"
    # beta_4 = "https://fwrest.senslabs.io/api/uploads/98-4"
    # delta = "https://fwrest.senslabs.io/api/uploads/98-5"
#     gamma = "https://fwrest.senslabs.io/api/uploads/98-6"
    # beta= "https://fwrest.senslabs.io/api/uploads/98-3"
#     production = "https://fwrest.senslabs.io/api/uploads/98-4"

    #################################!!!!!!!!!!!!CAREFUL!!!!!!!!!!!!!!##############################################
    beta_new = "https://daq-sit.dozee.cloud/api/uploads/0"
    new_production = "https://daq.dozee.cloud/api/uploads/0"
     
    
    basename =  os.path.basename(filename)
    # endpoint = f'{beta}/{device_id}/pzo/fsr/{basename}?force=true'
    
    endpoint = f'{beta_new}/{device_id}/pzo/{basename}'
    print(endpoint)
    ####################################!!!!!!!!!!!!!!!!!!CAREFUL!!!!!!!!##########################################
    # print(basename)
    with open(filename, 'rb') as f:
        data = f.read()
        r = requests.post(endpoint, headers=headers, data=data)
        print(r.text, r.status_code)

def start_reading(file_names, device_id, header_time_delta = 0):
    print("Total files =", len(file_names))
    print(f"Changing header time by {header_time_delta/60} mins Or {header_time_delta/3600} hrs")
    file_number = 0
    for file_name in file_names:
        try:
            raw_data, header_position, header_tstamp = read_data(file_name)
            print(file_name)
            print('Old Header Timestamp: {}'.format(header_tstamp))
            header_tstamp = header_tstamp +header_time_delta # + 10*60*60  # make change to the header time
            print('New Header Timestamp: {}'.format(header_tstamp))
            update_device_id_htime(raw_data, header_position, device_id, file_name, header_tstamp)
            file_number = file_number +1
              
            print(file_number)
        except Exception as e:
            print(file_name,"Exception",e)
            

In [4]:
int(time.time())

1723812761

In [5]:
----------------------------------------------CAREFUL AFTER THIS-----------------------------------------------------------------

SyntaxError: invalid syntax (2751526389.py, line 1)

Download raw files

In [5]:
# Device details
prefix = 'DP' 
sequence = 15790

#Indian standard time
From = "2024-08-06T10:30:00"    ##IST
To =   "2024-08-06T10:40:00"    ##IST


DeviceId = "33056008-da61-451d-bec5-30983c055f94"
print(DeviceId)
From =(dt.datetime.fromisoformat(From)-dt.timedelta(hours=5.5)).isoformat()
To =(dt.datetime.fromisoformat(To)-dt.timedelta(hours=5.5)).isoformat()
Emails = "deepsett@dozee.io"
get_raw_files(DeviceId, Emails, From , To)

33056008-da61-451d-bec5-30983c055f94


<Response [200]>

In [ ]:
TST-3399 - Prod
TST-3388 - SIT

#Apr 04
DOZ-8235 - Prod
DOZ-6765 - SIT



#Aug 05,2024
1. DP-9669 - C:\Users\vaishalib\Documents\SIT HR algo test\QA_Benchmarking_HR\QA_Benchmarking_HR\DZ_OSA_013\RF : 1722272839 to 1722301797
DP-9125: C:\Users\vaishalib\Documents\SIT HR algo test\QA_Benchmarking_HR\QA_Benchmarking_HR\DZ_BGS_IBP_149\7\pzo : 1721606503 to 1721692721
          C:\Users\vaishalib\Documents\SIT HR algo test\QA_Benchmarking_HR\QA_Benchmarking_HR\DZ_BGS_IBP_149\8\pzo :  1722038441 to 1722124700
         C:/Users/vaishalib/Documents/SIT HR algo test/QA_Benchmarking_HR/QA_Benchmarking_HR/DZ_BGS_IBP_149/9/pzo : 1722729620 to 1722815996
DP-9122 : DZ_OSA_013\RF: 1722366578,1. 1722704839 to 1722733797 2. 1721543485 to 1721606383
DP-7942 :C:\Users\vaishalib\Documents\SIT HR algo test\QA_Benchmarking_HR\QA_Benchmarking_HR\DZ_OSA_013\RF :1722704839 

Curl Files 

In [10]:

upload = True

prefix = 'DP' 
sequence = 9125
device_id = get_device_ID(prefix,sequence)
print('DeviceId:', device_id)

header_time_delta = int(3600*(24)*1) ## In seconds
print(header_time_delta)


if __name__ == '__main__':
  
    # file_names = sorted(glob.glob("/Users/mohit/Desktop/HDCS SIT/RAW_Files/*/*"))
    # file_names = sorted(glob.glob("/home/john/VScode/amit_reports/24hrs_data/12060000-14060000/*"))
    #filename = "C:/Users/vaishalib/Documents/SIT HR algo test/QA_Benchmarking_HR/QA_Benchmarking_HR/DZ_OSA_013/RF/*"
#     filename = "C:/Users/vaishalib/Documents/SIT HR algo test/QA_Benchmarking_HR/QA_Benchmarking_HR/DZ_BGS_IBP_149/6/pzo/*"
# filename = "C:/Users/vaishalib/Documents/SIT HR algo test/QA_Benchmarking_HR/QA_Benchmarking_HR/DZ_BGS_IBP_149/7/pzo/*"
#   filename = "C:/Users/vaishalib/Documents/SIT HR algo test/QA_Benchmarking_HR/QA_Benchmarking_HR/DZ_BGS_IBP_149/8/pzo/*"
    
    # IMPORTANT! Mention file path here....
    filename = "C:/Users/vaishalib/Documents/SIT HR algo test/QA_Benchmarking_HR/QA_Benchmarking_HR/DZ_BGS_IBP_149/9/pzo/*"
#     filename = filename.replace("\\","/") + "/*"
#     print(filename)                            
    file_names = sorted(glob.glob(filename))
    
    # file_names= file_names[0:1]

    device_id =  device_id
    header_time_delta = int(header_time_delta)
    
    #This is to update time
    if not upload:
        start_reading(file_names, device_id, header_time_delta) 
    
    ####################################################################
    
    

    if upload:
        for f in file_names:
            #Make sure the devices are not used for past 1-2 days. Patient should be paired to device before 3 days
            upload_file(get_device_ID('DP', 9125), '', f)
            # upload_file(get_device_ID('DOZ', 6765), '', f)
            # upload_file(get_device_ID('DP', 6081), '', f)

            # upload_file(get_device_ID('DOZ', 7986), '', f)
            # upload_file(get_device_ID('DOZ', 8129), '', f)

            # upload_file(get_device_ID('DOZ', 2678), '', f)
            # time.sleep(1) 
            pass

DeviceId: 814bc00b-696b-4204-bf62-5e8512917b5a
86400
https://daq-sit.dozee.cloud/api/uploads/0/814bc00b-696b-4204-bf62-5e8512917b5a/pzo/09121423
"OK"
 200
https://daq-sit.dozee.cloud/api/uploads/0/814bc00b-696b-4204-bf62-5e8512917b5a/pzo/09121623
"OK"
 200
https://daq-sit.dozee.cloud/api/uploads/0/814bc00b-696b-4204-bf62-5e8512917b5a/pzo/09121823
"OK"
 200
https://daq-sit.dozee.cloud/api/uploads/0/814bc00b-696b-4204-bf62-5e8512917b5a/pzo/09122023
"OK"
 200
https://daq-sit.dozee.cloud/api/uploads/0/814bc00b-696b-4204-bf62-5e8512917b5a/pzo/09122223
"OK"
 200
https://daq-sit.dozee.cloud/api/uploads/0/814bc00b-696b-4204-bf62-5e8512917b5a/pzo/09122423
"OK"
 200


In [ ]:
False

## Get Vitals

In [ ]:
headers = {
    # "Authorization": "Bearer xyz",
    "Content-Type": "application/json",
    "x-api-key": "4taVi1w0mS2rAG6UX4xJn8rwinotrGAz8wXzOiWj"
             }    

def get_vitals_values():

        url: str = "https://xrecords.gateway.senslabs.io/api/records/get"
            
        # Step 1: Enter UserId.
        user_id: str = "97ade1dd-a71b-41e3-8d1b-c8b3821e04c3"
            
        #Step 2: Change timestamp (format UTC)
        utc_from: str = "2024-04-15T02:00:00Z"
        utc_to: str = "2024-04-15T03:00:00Z"
            
            
        hr_conf: str = "HeartRateConfidence"
        br_conf: str = "BreathRateConfidence"
        hr: str = "HeartRate"
        br: str = "BreathRate"
        bps: str = "BpSys"
        bpd: str = "BpDia"
            
        #Step 4(If necessary): Change source
        source: str = "350"
        # utc_from = self.ist_time_string_to_utc(ist_time= str(from_ist_time))
        # utc_to = self.ist_time_string_to_utc(ist_time= str(to_ist_time))

        #userID, utc_from, utc_to
        response = requests.request(method='GET', 
                                    url=f"{url}?userId={user_id}&from={utc_from}&to={utc_to}&key={hr_conf}&key={br_conf}&key={hr}&key={br}&key={bps}&key={bpd}&Source={source}",
                                    headers= headers,
                                    timeout= 1
                                    )
        print("status:", response.status_code)
        print("Values:", response.json())
        
        #Step 5: Select location to save data.
        with open('C:/Users/vaishalib/Documents/SIT HR algo test/sanity_result_Prod1.json', 'w+', encoding='utf-8') as f:
            json.dump(response.json(), f, ensure_ascii=False, indent=4)

#         return response.json()
get_vitals_values()